# Face Recognition using Support Vector Classifier

    PCA and LDA are used to reduce dimensionality. This is followed by randomized search for optimizing the hyperparameters of SVC. 
    
    Results: Precision  Recall  F1
    With PCA  0.94      0.91      0.92
    With LDA  0.98      0.96      0.96

##  Import packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import cv2
import matplotlib.pyplot as plt
from time import time
import os

##  Prepare data

In [1]:
# Handle resizing of images, while maintaing aspect ratio
def resize_image(image, h, w):

    old_width = image.shape[1]
    old_height = image.shape[0]
    dH, dW = 0, 0
    
    if old_width < old_height:  #width is smaller, resize along width and compute new height
        aspect_r = w / old_width  #aspect ratio
        new_height = int(old_height * aspect_r)
        new_dim = (w, new_height)
        resized_img = cv2.resize(image, new_dim, interpolation = cv2.INTER_AREA)
        #dimensions of above image are: (new_height, desired_width). 
        #Since images in our dataset might be of diff. sizes, such resized images will end up being different sizes
        #hence crop the image as follows to make each image of size(desired_height, desired_width)
        dH = int((resized_img.shape[0] - h)/2.0)
        
    else:   #height is smaller, resize along height and compute new width
        aspect_r = h / old_height  #aspect ratio
        new_width = int(old_width * aspect_r)
        new_dim = (new_width, h)
        resized_img = cv2.resize(image, new_dim, interpolation = cv2.INTER_AREA)
        dW = int((resized_img.shape[1] - w)/2.0)
        
    new_h, new_w = resized_img.shape[:2]
    cropped_image = resized_img[dH:new_h - dH, dW:new_w - dW]  #dimensions of this image = desired dimensions
    #Make sure that the image is resized to desired dimensions
    return cv2.resize(cropped_image, (h, w), interpolation = cv2.INTER_AREA)  

#All images provided to the classifier should have fixed and equal sizes
dim = (64, 64)

print("Each image is resized to {}".format(dim))

Each image is resized to (64, 64)


In [4]:
def fetch_subject_images(path_to_subject, subject_number):
    X = np.array([])
    index = 0
    for subject_img in os.listdir(path_to_subject): #for each image in this subject's folder
        img_path = os.path.join(path_to_subject, subject_img)
        if img_path.endswith(".pgm") or img_path.endswith(".png") or img_path.endswith(".jpg") or img_path.endswith(".jpeg"):
            #Read image, convert it to grayscale and resize every image to a fixed size  
            img = cv2.imread(img_path, 0)
            img = resize_image(img, dim[0], dim[1]) 
            img_data = img.ravel()  #Flatten each image, so that each sample is a 1D vector
            X = img_data if not X.shape[0] else np.vstack((X, img_data))
            index += 1

    y = np.empty(index, dtype = int) #index = total no. of samples
    y.fill(subject_number)  #add labels
    return X, y

def fetch_data(dataset_path):

    # Get a the list of folder names in the path to dataset
    labels_list = [d for d in os.listdir(dataset_path) if "." not in str(d)]

    X = np.empty([0, dim[0]*dim[1]])
    y = np.empty([0])

    for i in range(0, len(labels_list)):  #for each person
        subject = str(labels_list[i])  #person i in list of ppl
        path_to_subject = os.path.join(dataset_path, subject) #full path to this person's directory
        
        #Read all images in this folder (all images of this person)
        X_, y_ = fetch_subject_images(path_to_subject, i)
        X = np.concatenate((X, X_), axis=0)
        y = np.append(y, y_)
        print("Subject {}, {} images loaded".format(i, X_.shape[0]))

    return X, y, labels_list

# Load training data 
dataset_path = "att_faces/"
X, y, labels_list_faces  = fetch_data(dataset_path)

Subject 0, 10 images loaded
Subject 1, 10 images loaded
Subject 2, 10 images loaded
Subject 3, 10 images loaded
Subject 4, 10 images loaded
Subject 5, 10 images loaded
Subject 6, 10 images loaded
Subject 7, 10 images loaded
Subject 8, 10 images loaded
Subject 9, 10 images loaded
Subject 10, 10 images loaded
Subject 11, 10 images loaded
Subject 12, 10 images loaded
Subject 13, 10 images loaded
Subject 14, 10 images loaded
Subject 15, 10 images loaded
Subject 16, 10 images loaded
Subject 17, 10 images loaded
Subject 18, 10 images loaded
Subject 19, 10 images loaded
Subject 20, 10 images loaded
Subject 21, 10 images loaded
Subject 22, 10 images loaded
Subject 23, 10 images loaded
Subject 24, 10 images loaded
Subject 25, 10 images loaded
Subject 26, 10 images loaded
Subject 27, 10 images loaded
Subject 28, 10 images loaded
Subject 29, 10 images loaded
Subject 30, 10 images loaded
Subject 31, 10 images loaded
Subject 32, 10 images loaded
Subject 33, 10 images loaded
Subject 34, 10 images lo

# Split into training and test sets

In [5]:
from sklearn.model_selection import train_test_split

n_classes = len(np.unique(y))
print("Dataset consists of {} samples and  {} classes".format(X.shape[0], n_classes))
print("Data: {} and labels: {}".format(X.shape, y.shape))                                                                                           
print(" ")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print("Training data {}, test data {}".format(X_train.shape, X_test.shape))

Dataset consists of 400 samples and  40 classes
Data: (400, 4096) and labels: (400,)
 
Training data (320, 4096), test data (80, 4096)


#  Principal Component Analysis (PCA)

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 0.95, whiten=True).fit(X_train)  #preserve 95% of variance
print("PCA reduced dimensionality from {} to {}".format(X_train.shape[1], pca.n_components_))

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print("New training and test data", X_train_pca.shape, "and", X_test_pca.shape)

PCA reduced dimensionality from 4096 to 118
New training and test data (320, 118) and (80, 118)


In [8]:
from time import time
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

svm_clf1 = SVC(kernel = "rbf", cache_size = 7000)
param_grid = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rand_search_cv = RandomizedSearchCV(svm_clf1, param_grid, n_iter = 10, verbose = 1)
#the search is cross-validated (defult 3 fold)

start = time()
rand_search_cv.fit(X_train_pca, y_train)
print("Randomized search took {} minutes".format((time() - start)/60.0))

print("Best estimator:")
print(rand_search_cv.best_estimator_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Randomized search took 0.03059083620707194 minutes
Best estimator:
SVC(C=10.265475491159425, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0035805796748390236,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.8s finished


## Classification report

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = rand_search_cv.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Classifier's accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Classifier's accuracy: 0.9125
             precision    recall  f1-score   support

        0.0       0.75      1.00      0.86         3
        1.0       1.00      1.00      1.00         1
        2.0       1.00      1.00      1.00         2
        3.0       1.00      1.00      1.00         4
        4.0       1.00      1.00      1.00         3
        5.0       1.00      1.00      1.00         3
        7.0       1.00      0.67      0.80         6
        8.0       1.00      1.00      1.00         2
        9.0       1.00      1.00      1.00         2
       10.0       1.00      1.00      1.00         2
       11.0       1.00      1.00      1.00         3
       12.0       0.00      0.00      0.00         2
       13.0       1.00      1.00      1.00         1
       14.0       1.00      1.00      1.00         3
       15.0       1.00      0.50      0.67         2
       17.0       1.00      1.00      1.00         3
       18.0       1.00      1.00      1.00         1
       19.0    

# Linear Discriminant Analysis (LDA)

In [32]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components = 118)

lda.fit(X_train, y_train)
X_train_lda = lda.transform(X_train)
X_test_lda = lda.transform(X_test)

n_comp = X_train_lda.shape[1]
print("LDA reduced dimensionality from {} to {}".format(X_train.shape[1], n_comp))
print("New training and test data", X_train_lda.shape, "and", X_test_lda.shape)

LDA reduced dimensionality from 4096 to 39
New training and test data (320, 39) and (80, 39)


In [33]:
from time import time
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

svm_clf2 = SVC(kernel = "rbf", cache_size = 7000)
param_grid = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rand_search_cv1 = RandomizedSearchCV(svm_clf2, param_grid, n_iter = 10, verbose = 1)

start = time()
rand_search_cv1.fit(X_train_lda, y_train)
print("Randomized search took {} minutes".format((time() - start)/60.0))

print("Best estimator:")
print(rand_search_cv1.best_estimator_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Randomized search took 0.018037609259287515 minutes
Best estimator:
SVC(C=1.4395175817157992, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0016982001626888103,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.0s finished


In [34]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = rand_search_cv1.predict(X_test_lda)
accuracy = accuracy_score(y_test, y_pred)
print("Classifier's accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Classifier's accuracy: 0.9625
             precision    recall  f1-score   support

        0.0       0.60      1.00      0.75         3
        1.0       1.00      1.00      1.00         1
        2.0       1.00      1.00      1.00         2
        3.0       1.00      1.00      1.00         4
        4.0       1.00      1.00      1.00         3
        5.0       1.00      1.00      1.00         3
        7.0       1.00      0.67      0.80         6
        8.0       1.00      1.00      1.00         2
        9.0       1.00      1.00      1.00         2
       10.0       1.00      1.00      1.00         2
       11.0       1.00      1.00      1.00         3
       12.0       1.00      1.00      1.00         2
       13.0       1.00      1.00      1.00         1
       14.0       1.00      1.00      1.00         3
       15.0       1.00      1.00      1.00         2
       17.0       1.00      1.00      1.00         3
       18.0       1.00      1.00      1.00         1
       19.0    